# Modeling Voter Turnout Behavior with Keras

In [1]:
import pandas as pd
import numpy as np
import pickle
import altair as alt
from sklearn.preprocessing import StandardScaler
from cats import dummify
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from modeling_functions import aggregate_samples, aggregate_errors, plot_agg_error
from texter import send

In [2]:
homesf = open('home_votes', 'rb')
homes = pickle.load(homesf)
homesf.close()
test_indicesf = open('split_indeces', 'rb')
split_indices = pickle.load(test_indicesf)
test_indicesf.close()

In [3]:
homes = pd.get_dummies(homes,columns=dummify)
extra_cats = [x for x in homes.columns if '_0' in x]
homes.drop(columns=extra_cats, inplace=True)

In [4]:
dropcols = ['v19pu','v20pu','vpu', 'pin']
X_train = homes[homes.index.isin(split_indices[2])].drop(columns=dropcols)
X_test = homes[~homes.index.isin(split_indices[2])].drop(columns=dropcols)
y_train_rv = homes[homes.index.isin(X_train.index)]['vpu']
y_train_19 = homes[homes.index.isin(X_train.index)]['v19pu']
y_train_20 = homes[homes.index.isin(X_train.index)]['v20pu']
y_test_rv = homes[~homes.index.isin(split_indices[2])]['vpu']
y_test_19 = homes[~homes.index.isin(split_indices[2])]['v19pu']
y_test_20 = homes[~homes.index.isin(split_indices[2])]['v20pu']

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
split = (X_train, X_test, y_train_rv, y_test_rv, y_train_19, y_test_19, y_train_20, y_test_20)
with open('nnsplits', 'wb') as file:
    pickle.dump(split, file)

In [ ]:
#tests = aggregate_samples(pd.DataFrame(X_test_pca), 100, 1000)

In [13]:
# compile and train RV regression on sample
model_rv = Sequential()
model_rv.add(Dense(512, activation= "relu"))
#model_rv.add(Dropout(.4))
model_rv.add(Dense(256, activation= "relu"))
#model_rv.add(Dropout(.4))
model_rv.add(Dense(128, activation= "relu"))
model_rv.add(Dense(64, activation="relu"))
model_rv.add(Dense(1))
model_rv.compile(
    loss= "mean_squared_error", optimizer="adam"
)

In [12]:
history_rv = model_rv.fit(np.array(X_train).astype('float32'), np.array(y_train_rv), epochs=10, validation_split=.2)

Train on 572175 samples, validate on 143044 samples
Epoch 1/10
572175/572175 [==============================] - 174s 303us/sample - loss: 31502.3850 - val_loss: 96213.2969
Epoch 2/10
572175/572175 [==============================] - 175s 306us/sample - loss: 79637.2585 - val_loss: 1129177.5914
Epoch 3/10
572175/572175 [==============================] - 165s 289us/sample - loss: 2556146.4810 - val_loss: 6709.0847
Epoch 4/10
572175/572175 [==============================] - 174s 304us/sample - loss: 106632657.4752 - val_loss: 5219011.3330
Epoch 5/10
572175/572175 [==============================] - 160s 279us/sample - loss: 709252038.9011 - val_loss: 2500328889.8568
Epoch 6/10
572175/572175 [==============================] - 166s 291us/sample - loss: 1572604271.8582 - val_loss: 39557956774.5844
Epoch 7/10
572175/572175 [==============================] - 163s 285us/sample - loss: 5215294394.2190 - val_loss: 556155463.5993
Epoch 8/10
572175/572175 [==============================] - 167s 292us

KeyboardInterrupt: 

In [8]:
history_rv2 = model_rv.fit(np.array(X_train).astype('float32'), np.array(y_train_rv), epochs=15, validation_split=.2)

Train on 572175 samples, validate on 143044 samples
Epoch 1/15
572175/572175 [==============================] - 215s 376us/sample - loss: 1.3177 - val_loss: 0.7054
Epoch 2/15
572175/572175 [==============================] - 284s 496us/sample - loss: 0.8422 - val_loss: 0.6794
Epoch 3/15
572175/572175 [==============================] - 271s 473us/sample - loss: 0.9447 - val_loss: 0.6742
Epoch 4/15
572175/572175 [==============================] - 227s 396us/sample - loss: 2.0491 - val_loss: 0.6833
Epoch 5/15
572175/572175 [==============================] - 225s 394us/sample - loss: 0.9794 - val_loss: 0.9883
Epoch 6/15
572175/572175 [==============================] - 226s 395us/sample - loss: 0.8155 - val_loss: 0.6815
Epoch 7/15
572175/572175 [==============================] - 226s 394us/sample - loss: 0.9762 - val_loss: 0.7114
Epoch 8/15
572175/572175 [==============================] - 234s 410us/sample - loss: 1.1014 - val_loss: 0.6851
Epoch 9/15
572175/572175 [==========================

KeyboardInterrupt: 

In [24]:
rv_pred = model_rv.predict(np.array(X_test).astype('float32'))

In [25]:
rv_pred.sum()

359966.3

In [26]:
y_test_rv.sum()

347631.4478521943

In [11]:
model_rv.save('nnregrv')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: nnregrv/assets


In [12]:
send('RV Network Done')

## 2019 Voters

In [110]:
model_19 = Sequential()
#model.add(Dense(500, input_dim=4, activation= "relu"))
model_19.add(Dense(100, input_dim=431, activation= "relu"))
model_19.add(Dropout(.6))
model_19.add(Dense(50, activation= "relu"))
model_19.add(Dense(1))
model_19.compile(
    loss= "mean_squared_error", optimizer="adam"
)

In [111]:
history19 = model_19.fit(np.array(X_train).astype('float32'), np.array(y_train_19), epochs=70,  validation_split=.2)

Train on 572175 samples, validate on 143044 samples
Epoch 1/70
572175/572175 [==============================] - 90s 158us/sample - loss: 0.6158 - val_loss: 0.6444
Epoch 2/70
572175/572175 [==============================] - 93s 163us/sample - loss: 0.4610 - val_loss: 0.6273
Epoch 3/70
572175/572175 [==============================] - 93s 163us/sample - loss: 0.4453 - val_loss: 0.6353
Epoch 4/70
572175/572175 [==============================] - 94s 164us/sample - loss: 0.4588 - val_loss: 0.6324
Epoch 5/70
572175/572175 [==============================] - 94s 164us/sample - loss: 0.4370 - val_loss: 0.6301
Epoch 6/70
572175/572175 [==============================] - 94s 164us/sample - loss: 0.4501 - val_loss: 0.6383
Epoch 7/70
572175/572175 [==============================] - 94s 164us/sample - loss: 0.4441 - val_loss: 0.6320
Epoch 8/70
572175/572175 [==============================] - 94s 165us/sample - loss: 0.4237 - val_loss: 0.6381
Epoch 9/70
572175/572175 [==============================] - 

KeyboardInterrupt: 

In [ ]:
model_19.save('nnreg19')

In [ ]:
send('2019 Network Done')

## 2020 Voters

In [35]:
# compile and train RV regression on sample
model_20 = Sequential()
model_20.add(Dense(500, input_dim=431, activation= "relu"))
model_20.add(Dropout(.3))
model_20.add(Dense(100, activation= "relu"))
model_20.add(Dense(50, activation= "relu"))
model_20.add(Dense(1))
model_20.compile(
    loss= "mean_squared_error", optimizer="adam"
)

In [36]:
history20 = model_20.fit(np.array(X_train).astype('float32'), np.array(y_train_20), epochs=10, validation_split=.2)

Train on 572175 samples, validate on 143044 samples
Epoch 1/10
572175/572175 [==============================] - 174s 303us/sample - loss: 0.8601 - val_loss: 0.7272
Epoch 2/10
572175/572175 [==============================] - 176s 308us/sample - loss: 0.6928 - val_loss: 0.6797
Epoch 3/10
572175/572175 [==============================] - 178s 311us/sample - loss: 0.7932 - val_loss: 0.6762
Epoch 4/10
572175/572175 [==============================] - 176s 308us/sample - loss: 0.7000 - val_loss: 0.6644
Epoch 5/10
572175/572175 [==============================] - 181s 316us/sample - loss: 0.7731 - val_loss: 0.6651
Epoch 6/10
572175/572175 [==============================] - 183s 319us/sample - loss: 0.6492 - val_loss: 0.6614
Epoch 7/10
572175/572175 [==============================] - 179s 313us/sample - loss: 0.6145 - val_loss: 0.6586
Epoch 8/10
572175/572175 [==============================] - 177s 310us/sample - loss: 0.5498 - val_loss: 0.6623
Epoch 9/10
106752/572175 [====>.....................

KeyboardInterrupt: 

In [ ]:
model_20.save('nnreg20')

In [ ]:
send('2020 Network Done')